In [2]:
import os

import geopandas as gpd
import pandas as pd
from config import TAGS
from shapely import geometry
from srai.constants import REGIONS_INDEX, WGS84_CRS
from srai.embedders import CountEmbedder
from srai.joiners import IntersectionJoiner
from srai.loaders.osm_loaders import OSMOnlineLoader, OSMPbfLoader
from srai.regionizers import H3Regionizer, SlippyMapRegionizer
from srai.utils import geocode_to_region_gdf

In [2]:

def get_embedding_for_region(region_name: str) -> pd.DataFrame:
    bbox_gdf = geocode_to_region_gdf(region_name)
    regionizer = SlippyMapRegionizer(zoom=16)
    regions_gdf = regionizer.transform(bbox_gdf)
    loader = OSMPbfLoader()
    features_gdf = loader.load(bbox_gdf, tags=TAGS)
    joiner = IntersectionJoiner()
    joint_gdf = joiner.transform(regions_gdf.drop(columns=["x", "y", "z"]), features_gdf)
    embedder = CountEmbedder()
    embedding = embedder.transform(regions_gdf.drop(columns=["x", "y", "z"]), features_gdf, joint_gdf)
    return embedding

In [4]:
with open('../data/cities.txt', 'r') as cities_file:
    for city in cities_file:
        print(city)
        city = city.replace("\n", "")
        path = f"../data/embeddings2/{city}.pkl"
        df = get_embedding_for_region(city)
        df.to_pickle(path)

Wroclaw, Poland


[Wrocław, Lower Silesian Voivodeship, Poland] Parsing pbf file #1: : 2855433it [02:10, 21865.71it/s] 


In [ ]:
pd.read_pickle('data/embeddings/Tirana, Albania.pkl')

,aeroway_hangar,aeroway_helipad,amenity_arts_centre,amenity_atm,amenity_bank,amenity_bar,amenity_bbq,amenity_bench,amenity_bicycle_parking,amenity_bicycle_rental,...,tourism_museum,tourism_picnic_site,tourism_viewpoint,water_lake,water_pond,water_reservoir,water_river,waterway_canal,waterway_river,waterway_stream
region_id,,,,,,,,,,,,,,,,,,,,,
"(36364, 24482)",0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(36365, 24482)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
"(36366, 24482)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(36364, 24483)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(36365, 24483)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(36377, 24499)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(36378, 24499)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(36379, 24499)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
folium_map = plot_regions(get_embedding_for_region("Wrocław, PL"), tiles_style="CartoDB positron", colormap=["rgba(0,0,0,0)"])
df.reset_index().explore(m=folium_map, column=REGIONS_INDEX, cmap=px.colors.qualitative.Bold)

# Admin level names

In [ ]:
class AdminLevels:
    def __init__(self, gdf: gpd.GeoDataFrame) -> None:
        self.admins: dict[int, gpd.GeoDataFrame] = {}
        self.region_gdf = gdf
        self.names: dict[str, str] = {}


def get_admin_level(poligon: shapely.Polygon, regions: AdminLevels):
    for i in range(11, 7, -1):
        if not i in regions.admins:
            try:
                print(f"Getting admin level {i}")
                region_admins_i = get_regionazer(i, regions)
                region_admins_i["name"] = region_admins_i.apply(
                    lambda x: get_place_name(x),
                    axis=1,
                )
                regions.admins[i] = region_admins_i
            except ValueError as e:
                regions.admins[i] = gpd.GeoDataFrame().set_geometry(pd.Series())
        intersections = regions.admins[i].geometry.intersects(poligon)
        if intersections.any():
            district = intersections[intersections].sample()
            row = regions.admins[i].loc[district.index].sample()
            return row.reset_index().iloc[0]
    return pd.Series(
        ["unknown", None, "unknown"], index=["region_id", "geometry", "name"]
    )


def get_regionazer(i, regions):
    try:
        regionazer = AdministrativeBoundaryRegionizer(i, toposimplify=1e-2)
        return regionazer.transform(regions.region_gdf)

    except RuntimeError:  # lower toposimplify
        regionazer = AdministrativeBoundaryRegionizer(i)
        return regionazer.transform(regions.region_gdf)


def get_place_name(x):
    try:
        places = osmnx.geometries_from_polygon(
            geocode_to_region_gdf(x.name).geometry.tolist()[0],
            {"place": True},
        ).reset_index()
        return "unknown" if places.shape[0] == 0 else places.iloc[0]["place"]
    except Exception as e:
        print(x.name)
        raise e


def process(geocode: str):
    city = geocode_to_region_gdf(geocode)
    r = SlippyMapRegionizer(zoom=16)
    tiles = r.transform(gdf=city)
    regions = AdminLevels(city)
    regions.admins[8] = AdministrativeBoundaryRegionizer(8).transform(city)
    tiles = tiles.progress_apply(
        lambda row: get_admin_level(r._polygon_from_x_y(row["x"], row["y"]), regions),
        axis=1,
    )
    try:
        return tiles.rename(columns={"region_id": "admin_region"})
    except TypeError:
        print("eee")

In [ ]:
exists = seq(os.listdir("../data/admins")).map(lambda x: x.replace(".pkl", "")).to_set()
for city in tqdm(os.listdir("../data/tiles")):
    if city not in exists:
        print(city)
        data = process(city)
        data.to_pickle(f"../data/admins/{city}.pkl")
    else:
        print(city, "skipped")

data

In [8]:
pds = []
for file in os.listdir("../data/admins"):
    cdf = pd.read_pickle(f"../data/admins/{file}")
    pds.append(cdf)
    print(file)
    display(cdf)
df = pd.concat(pds)
df

Philadelphia, USA.pkl


,geometry,name,admin_region
region_id,,,
19063_24816_16,"POLYGON ((-75.27648 39.97698, -75.27123 39.979...",NaN,Philadelphia
19063_24817_16,"POLYGON ((-75.27648 39.97698, -75.27123 39.979...",NaN,Philadelphia
19064_24815_16,"POLYGON ((-75.27648 39.97698, -75.27123 39.979...",NaN,Philadelphia
19064_24816_16,"POLYGON ((-75.27648 39.97698, -75.27123 39.979...",NaN,Philadelphia
19064_24817_16,"POLYGON ((-75.27648 39.97698, -75.27123 39.979...",NaN,Philadelphia
...,...,...,...
19122_24787_16,"POLYGON ((-75.27648 39.97698, -75.27123 39.979...",NaN,Philadelphia
19122_24788_16,"POLYGON ((-75.27648 39.97698, -75.27123 39.979...",NaN,Philadelphia
19122_24789_16,"POLYGON ((-75.27648 39.97698, -75.27123 39.979...",NaN,Philadelphia


Skopje, North Macedonia.pkl


,geometry,name,admin_region
region_id,,,
36648_24322_16,"POLYGON ((21.31788 41.99857, 21.31839 41.99988...",NaN,Skopje
36648_24323_16,"POLYGON ((21.31788 41.99857, 21.31839 41.99988...",NaN,Skopje
36648_24327_16,"POLYGON ((21.31788 41.99857, 21.31839 41.99988...",NaN,Skopje
36648_24328_16,"POLYGON ((21.34733 42.00229, 21.31682 41.99270...",village,Saraj
36648_24329_16,"POLYGON ((21.34733 42.00229, 21.31682 41.99270...",village,Saraj
...,...,...,...
36687_24335_16,"POLYGON ((21.44986 41.99475, 21.52070 41.97575...",suburb,Аеродром
36687_24336_16,"POLYGON ((21.44986 41.99475, 21.52070 41.97575...",suburb,Аеродром
36687_24337_16,"POLYGON ((21.31788 41.99857, 21.31839 41.99988...",NaN,Skopje


Dublin, Ireland.pkl


...

Poznań, PL.pkl


,admin_region,geometry,name
region_id,,,
35813_21509_16,Kiekrz,"POLYGON ((16.81051 52.47225, 16.80214 52.47238...",suburb
35813_21510_16,Kiekrz,"POLYGON ((16.81051 52.47225, 16.80214 52.47238...",suburb
35814_21509_16,Kiekrz,"POLYGON ((16.81051 52.47225, 16.80214 52.47238...",suburb
35814_21510_16,Kiekrz,"POLYGON ((16.81051 52.47225, 16.80214 52.47238...",suburb
35814_21511_16,Kiekrz,"POLYGON ((16.81051 52.47225, 16.80214 52.47238...",suburb
...,...,...,...
35874_21523_16,Antoninek-Zieliniec-Kobylepole,"POLYGON ((16.98958 52.40231, 16.98977 52.40246...",suburb
35874_21524_16,Antoninek-Zieliniec-Kobylepole,"POLYGON ((16.98958 52.40231, 16.98977 52.40246...",suburb
35875_21520_16,Antoninek-Zieliniec-Kobylepole,"POLYGON ((16.98958 52.40231, 16.98977 52.40246...",suburb


Chișinău, Moldova.pkl


,admin_region,geometry
region_id,,
38000_23032_16,Chișinau,"POLYGON ((28.75219 47.07163, 28.75233 47.07169..."
38000_23033_16,Chișinau,"POLYGON ((28.75219 47.07163, 28.75233 47.07169..."
38001_23032_16,Chișinau,"POLYGON ((28.75219 47.07163, 28.75233 47.07169..."
38001_23033_16,Chișinau,"POLYGON ((28.75219 47.07163, 28.75233 47.07169..."
38001_23034_16,Chișinau,"POLYGON ((28.75219 47.07163, 28.75233 47.07169..."
...,...,...
38041_23068_16,Chișinau,"POLYGON ((28.75219 47.07163, 28.75233 47.07169..."
38041_23069_16,Chișinau,"POLYGON ((28.75219 47.07163, 28.75233 47.07169..."
38041_23070_16,Chișinau,"POLYGON ((28.75219 47.07163, 28.75233 47.07169..."


,geometry,name,admin_region
region_id,,,
19063_24816_16,"POLYGON ((-75.27648 39.97698, -75.27123 39.979...",NaN,Philadelphia
19063_24817_16,"POLYGON ((-75.27648 39.97698, -75.27123 39.979...",NaN,Philadelphia
19064_24815_16,"POLYGON ((-75.27648 39.97698, -75.27123 39.979...",NaN,Philadelphia
19064_24816_16,"POLYGON ((-75.27648 39.97698, -75.27123 39.979...",NaN,Philadelphia
19064_24817_16,"POLYGON ((-75.27648 39.97698, -75.27123 39.979...",NaN,Philadelphia
...,...,...,...
38041_23068_16,"POLYGON ((28.75219 47.07163, 28.75233 47.07169...",NaN,Chișinau
38041_23069_16,"POLYGON ((28.75219 47.07163, 28.75233 47.07169...",NaN,Chișinau
38041_23070_16,"POLYGON ((28.75219 47.07163, 28.75233 47.07169...",NaN,Chișinau


In [15]:
df["name"].value_counts(dropna=False)

name
NaN                  30188
suburb               27347
unknown              10348
neighbourhood         7202
quarter               5158
locality              4680
hamlet                4246
village               3150
city                  2764
town                  2695
square                 618
farm                   338
district               217
county                 194
island                 193
isolated_dwelling      107
city_block              38
Name: count, dtype: int64